In [36]:
import pandas as pd
import numpy as np
import joblib
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
import re

In [37]:
data = pd.read_csv('test_data.csv',encoding='utf-8')#读取文件
passport = data['pax_passport']

/Users/hanpeitao/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (46,47,56,57,141,142,266,340,360,437,650,651) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [38]:
data['seg_dep_time_loc']=pd.to_datetime(data['seg_dep_time'])
print(data['seg_dep_time'])
max_date=pd.Timestamp(data['seg_dep_time_loc'].max()).timestamp()
date_df=data['seg_dep_time_loc'].apply(lambda x:pd.Timestamp(x).timestamp())
data['seg_dep_time_interval']=data['seg_dep_time_loc'].apply(lambda x:x.hour)
data.drop('seg_dep_time',axis=1,inplace=True)
data['seg_dep_time_loc']=max_date-date_df
data['seg_dep_time_gap']=data['seg_dep_time_loc']/(24*60*60)

0       2019-04-10 11:30:00
1       2019-12-04 21:00:00
2       2018-11-29 11:30:00
3       2017-12-20 13:00:00
4       2017-10-24 19:10:00
               ...         
6766    2019-11-05 21:00:00
6767    2019-07-04 21:05:00
6768    2020-02-23 20:20:00
6769    2019-09-24 11:30:00
6770    2019-07-04 21:05:00
Name: seg_dep_time, Length: 6771, dtype: object


* 1) seg_dep_time_gap               0.237879
* 2) pax_fcny                       0.230680
* 3) seg_dep_time_interval          0.042140
* 4) seg_cabin                      0.041153
* 5) flt_bag_cnt_y1                 0.037611
* 6) avg_dist_cnt_y1                0.035616

In [39]:
data2=data[['seg_dep_time_gap','pax_fcny','seg_dep_time_interval','seg_cabin']]

In [40]:
data2

,seg_dep_time_gap,pax_fcny,seg_dep_time_interval,seg_cabin
0,602.395833,339.0,11,Y
1,364.000000,255.0,21,Y
2,734.395833,479.0,11,Y
3,1078.333333,400.0,13,Y
4,1135.076389,500.0,19,Y
...,...,...,...,...
6766,393.000000,2707.0,21,J
6767,516.996528,216000.0,21,Y
6768,283.027778,324.0,20,Y
6769,435.395833,385.0,11,Y


In [41]:
flt_col = []
for col in data.columns:
    res_y3 = re.search(r'^flt_bag_cnt',col)
    if res_y3:
        #print(res.group())
        flt_col.append(col)
flt_df=pd.DataFrame(data[flt_col])
for i in range(flt_df.shape[0]):
    for j in range(flt_df.shape[1]-1):
        if flt_df.iloc[i,j+1]<flt_df.iloc[i,j]:
            flt_df.iloc[i,j+1]=flt_df.iloc[i,j]
res3=flt_df[flt_col[0]]*0.6+(flt_df[flt_col[1]]-flt_df[flt_col[0]])*0.3+(flt_df[flt_col[2]]-flt_df[flt_col[1]])*0.1
data2['flt_bag_cnt_y1']=res3

/Users/hanpeitao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [42]:
flt_col = []
for col in data.columns:
    res_y3 = re.search(r'^avg_dist_cnt',col)
    if res_y3:
        #print(res.group())
        flt_col.append(col)
flt_df=pd.DataFrame(data[flt_col])
for i in range(flt_df.shape[0]):
    for j in range(flt_df.shape[1]-1):
        if flt_df.iloc[i,j+1]<flt_df.iloc[i,j]:
            flt_df.iloc[i,j+1]=flt_df.iloc[i,j]
res3=flt_df[flt_col[0]]*0.6+(flt_df[flt_col[1]]-flt_df[flt_col[0]])*0.3+(flt_df[flt_col[2]]-flt_df[flt_col[1]])*0.1
data2['avg_dist_cnt_y1']=res3

/Users/hanpeitao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [43]:
data2

,seg_dep_time_gap,pax_fcny,seg_dep_time_interval,seg_cabin,flt_bag_cnt_y1,avg_dist_cnt_y1
0,602.395833,339.0,11,Y,0.0,2880.0
1,364.000000,255.0,21,Y,0.0,0.0
2,734.395833,479.0,11,Y,0.0,410.8
3,1078.333333,400.0,13,Y,0.0,0.0
4,1135.076389,500.0,19,Y,0.0,0.0
...,...,...,...,...,...,...
6766,393.000000,2707.0,21,J,0.0,0.0
6767,516.996528,216000.0,21,Y,0.0,0.0
6768,283.027778,324.0,20,Y,0.0,0.0
6769,435.395833,385.0,11,Y,0.0,0.0


In [44]:
object_col=['seg_cabin']
for col in object_col:
    dict_t = np.load('dict/dict_{}.npy'.format(col),allow_pickle=True).item()
    dict_now = data2[col].value_counts(1).to_dict()
    #print(dict_t,dict_now)
    for i in dict_now:
        print(i)
        if i not in dict_t:
            dict_t=0
    data2[col].replace(dict_t,inplace=True)
data2 = data2.iloc[:, :2]

Y
J
F


/Users/hanpeitao/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [45]:
data2 = (data2-data2.mean())/data2.std()
data2 = (data2-data2.min())/(data2.max()-data2.min())

In [46]:
model = joblib.load('a04.model')
y_pred = model.predict(data2) # 对新的样本Z做预测 
y_ab = model.predict_proba(data2)

In [49]:
df=pd.DataFrame(y_ab)
data2['y_pred']=y_pred
data2[['y_ab_0','y_ab_1']]=df

In [33]:
data2['passport']=passport
data2.drop(['seg_dep_time_gap','pax_fcny'],axis=1,inplace=True)

In [92]:
data2.to_csv('res.csv',index=False)